In [62]:
import httpx
from bs4 import BeautifulSoup

url = "https://www.goodreads.com/review/list/96888-mary?shelf=read"

def get_response(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    with httpx.Client(headers=headers, follow_redirects=True) as client:
        response = client.get(url)
        if response.status_code == 200:
            return response.text
        elif response.status_code == 404:
            raise ValueError(f"404 Not Found: {response.url}")
        else:
            raise ValueError(f"Unexpected status: {response.status_code}")

In [70]:
import re
from datetime import datetime
from calendar import monthrange
import pandas as pd

html = get_response(url)
soup = BeautifulSoup(html, "html.parser")
# Find all <tr> tags with id starting with 'review_'
review_trs = soup.find_all('tr', id=re.compile(r'^review_'))
# Extract date strings and review text
data = []
for tr in review_trs:
    # Date extraction
    date_span = tr.find('span', class_='date_read_value')
    date_str = date_span.get_text(strip=True) if date_span else None
    # Review extraction
    review_span = tr.find('span', id=re.compile(r'^freeTextContainerreview'))
    review_text = review_span.get_text(strip=True) if review_span else None
    data.append({'date': date_str, 'review': review_text})

def parse_date(date_str):
    for fmt in ("%b %d, %Y", "%b %Y"):
        try:
            dt = datetime.strptime(date_str, fmt)
            if fmt == "%b %Y":
                # Set to last day of the month
                last_day = monthrange(dt.year, dt.month)[1]
                dt = dt.replace(day=last_day)
            return dt
        except Exception:
            continue
    return None

# Parse dates and build DataFrame
for row in data:
    row['parsed_date'] = parse_date(row['date'])

df = pd.DataFrame(data)
print(df[['parsed_date', 'review']])


/Users/jkazimer1/Documents/GitHub/PyReads/.venv/lib/python3.12/site-packages/numpy/lib/array_utils.py:1: RuntimeWarning: coroutine 'get_html' was never awaited
  from ._array_utils_impl import (  # noqa: F401


   parsed_date                                             review
0   2010-12-18  I'd love to be a whole lot more like Ruth Reic...
1   2010-10-31                                             Bleak!
2   2010-12-02  I can't believe I slogged through the whole th...
3   2010-09-19  Good golly. There was so much promise here: th...
4   2010-09-03  **spoiler alert** Hm. I'm not sure what to say...
5   2010-08-30  What I like about The Hunger Games was how it ...
6   2010-08-24  Engaging, but just not emotional. The writing ...
7   2010-07-22  Very fun, very McMurtry, and therefore very ea...
8   2010-05-18  I'm sad that I've already forgotten all of the...
9   2010-04-30  A helpful chapter-by-chapter overview for a fi...
10  2010-04-30  It's hard to give a rating to this book. I lik...
11  2010-02-15  I was preparing for a trip to visit Seattle fo...
12  2010-02-03  It's hard to rate The Odyssey. It's like a mor...
13  2010-01-06  Once for an MTV awards show, Ben Stiller playe...
14  2009-1